# Day 1

In [2]:
def two_sum(target, nums):
    data = set([target - val for val in nums])
    for val in nums:
        if val in data:
            print(val, target - val)

In [3]:
with open("p1_input") as fd:
    nums = [int(line.strip()) for line in fd]
    
two_sum(2020, nums)

1504 516
516 1504


In [4]:
1504 * 516

776064

In [5]:
def two_sum_v2(target, val, begin, nums, res):
    target -= val
    first, second = begin, len(nums) - 1
    while first < second:
        if nums[first] + nums[second] == target:
            res.append([val, nums[first], nums[second]])
            first += 1
            second -= 1
        elif nums[first] + nums[second] < target:
            first += 1
        elif nums[first] + nums[second] > target:
            second -= 1

def three_sum(target, nums):
    nums.sort()
    res = []
    for i, first in enumerate(nums):
        two_sum_v2(target, first, i + 1, nums, res)
    return res
        
res = three_sum(2020, nums)
print(res)
from functools import reduce
print(sum(res[0]), reduce(lambda x, y: x*y, res[0]))

[[13, 317, 1690]]
2020 6964490


In [6]:
13 * 317 * 1690

6964490

# Day 2

In [7]:
def valid_password(rule, pwd):
    cnt, symb = rule.split()
    min_cnt, max_cnt = map(int, cnt.split("-"))
    stats = {}
    for c in pwd:
        stats[c] = stats.get(c, 0) + 1
    return min_cnt <= stats.get(symb, 0) <= max_cnt


def valid_password2(rule, pwd):
    cnt, symb = rule.split()
    first, second = map(int, cnt.split("-"))
    count = int(pwd[first - 1] == symb) + int(pwd[second - 1] == symb)
    return count == 1


filename = "day2_passwords.txt"
data = [line.strip() for line in open(filename)]
cnt, cnt2 = 0, 0
for rec in data:
    rule, pwd = rec.split(":")
    cnt += int(valid_password(rule, pwd.strip()))
    cnt2 += int(valid_password2(rule, pwd.strip()))

print(cnt, cnt2)

493 593


# Day 3

In [8]:
def count_trees(board, dx=3, dy=1, tree="#"):
    maxx = len(board[0])
    maxy = len(board)
    x, y = 0, 0
    cnt = 0
    while y + dy < maxy:
        x = (x + dx) % maxx
        y += dy
        cnt += int(board[y][x] == tree)
    return cnt

filename = "day3_trees.txt"
data = []
with open(filename) as fd:
    for line in fd:
        data.append(list(line.strip()))

res = count_trees(data)
print(res)

slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)] 
res = [count_trees(data, dx, dy) for dx, dy in slopes]
print(res)
import math
print(math.prod(res))

214
[94, 214, 99, 91, 46]
8336352024


# Day 4

In [9]:
import re

MANDATORY_KEYS = set("byr iyr eyr hgt hcl ecl pid".split())
def valid_pass(passport, mandatory):
    return mandatory.issubset(set(passport))


def count_passports(filename, validate_func):
    with open(filename) as fd:
        current = {}
        cnt = 0
        for line in fd:
            line = line.strip()
            if not line:
                cnt += int(validate_func(current, MANDATORY_KEYS))
                current = {}
                continue
            current.update({rec.split(":")[0]:rec.split(":")[1] for rec in line.split()})
    if current:
        cnt += int(validate_func(current, MANDATORY_KEYS))
    return cnt


print(count_passports("day4_test.txt", valid_pass))
print(count_passports("day4_passports.txt", valid_pass))
                

class IntType:
    def __init__(self, val=None, min_val=None, max_val=None, num_digits=None):
        self.min_val = min_val
        self.max_val = max_val
        self.num_digits = num_digits
        self.val = val
        
    def init(self, val):
        self.validate_init = True
        self.val = val
        return self
    
    def validate(self):
        if not self.validate_init:
            return False
        if self.num_digits and len(self.val) < self.num_digits:
            return False
        val = int(self.val)
        if self.min_val and val < self.min_val:
            return False
        if self.max_val and val > self.max_val:
            return False
        return True


class HeightType(IntType):
    def init(self, val):
        self.validate_init = True
        if val.endswith("cm"):
            self.min_val, self.max_val = 150, 193
            self.val = val[:-2]
        elif val.endswith("in"):
            self.min_val, self.max_val = 59, 76
            self.val = val[:-2]
        else:
            self.validate_init = False
        #print(self.val)
        return self


class PatternType:
    def __init__(self, pattern):
        self.pattern = re.compile(pattern)
    def init(self, val):
        self.val = val
        return self
    def validate(self):
        res = self.pattern.match(self.val)
        return bool(res)

class AnyType:
    def init(self, val):
        return self
    def validate(self):
        return True

    
birth_year = IntType(min_val=1920, max_val=2002, num_digits=4)
issue_year = IntType(min_val=2010, max_val=2020, num_digits=4)
exp_year = IntType(min_val=2020, max_val=2030, num_digits=4)
height = HeightType()
hair_color = PatternType(r'^\#(\d|[a-f]){6}$')
eye_color = PatternType(r'^((amb)|(blu)|(brn)|(gry)|(grn)|(hzl)|(oth))$')
passport_id = PatternType(r'^\d{9}$')
country_id = AnyType()


VALIDATOR_MAP = {
    "byr": birth_year,
    "iyr": issue_year,
    "eyr": exp_year,
    "hgt": height,
    "hcl": hair_color,
    "ecl": eye_color,
    "pid": passport_id,
    "cid": country_id
}


def valid_pass_v2(passport, mandatory):
    if not mandatory.issubset(set(passport)):
        return False
    #print(passport)
    for key in mandatory:
        res = VALIDATOR_MAP[key].init(passport[key]).validate()
        if not res:
            return False
    return True

2
256


In [10]:
tests = "byr,2002 byr,2003 hgt,74in hgt,60in hgt,190cm hgt,300cm hgt,200 hgt,190in hcl,#123abc hcl,#123abz hcl,123abc ecl,brn ecl,vat pid,000000001 pid,0123456789 cid,2e23r cid,999"
for t in tests.split():
    cls, val = t.split(",")
    res = VALIDATOR_MAP[cls].init(val).validate()
    #print(cls, val, res)

In [11]:
print(count_passports("day4_valid_test.txt", valid_pass_v2))
print(count_passports("day4_invalid_test.txt", valid_pass_v2))
print(count_passports("day4_passports.txt", valid_pass_v2))


4
0
198


# Day 5

In [29]:
def binsearch(code, num_elements, first, second):
    minv, maxv = 0, num_elements - 1
    for c in code:
        mid = minv + (maxv - minv + 1) / 2
        if c == first:
            maxv = mid - 1
        else:
            minv = mid
        #print(c, mid, minv, maxv)
    return minv


def get_seat_id(code):
    row_code = code[:7]
    col_code = code[7:]
    row = binsearch(row_code, 128, "F", "B")
    col = binsearch(col_code, 8, "L", "R")
    return int(row), int(col), int(8 * row + col


In [30]:
tests = [("BFFFBBFRRR", 567),
         ("FFFBBBFRRR", 119),
         ("BBFFBBFRLL", 820)]

for code, expected in tests:
    row, col, seat_id = get_seat_id(code)
    print(code, row, col, seat_id, expected)

BFFFBBFRRR 70.0 7.0 567.0 567
FFFBBBFRRR 14.0 7.0 119.0 119
BBFFBBFRLL 102.0 4.0 820.0 820


In [31]:
filename = "day5_seats.txt"
data = [line.strip() for line in open(filename)]
seat_ids = [get_seat_id(code)[2] for code in data]


In [32]:
seat_ids

[192.0,
 681.0,
 639.0,
 320.0,
 94.0,
 409.0,
 471.0,
 352.0,
 343.0,
 130.0,
 346.0,
 281.0,
 496.0,
 832.0,
 490.0,
 495.0,
 312.0,
 257.0,
 646.0,
 570.0,
 492.0,
 701.0,
 707.0,
 783.0,
 183.0,
 171.0,
 878.0,
 851.0,
 483.0,
 698.0,
 644.0,
 789.0,
 795.0,
 374.0,
 690.0,
 828.0,
 826.0,
 261.0,
 342.0,
 425.0,
 712.0,
 533.0,
 387.0,
 111.0,
 291.0,
 217.0,
 361.0,
 621.0,
 255.0,
 308.0,
 548.0,
 811.0,
 557.0,
 629.0,
 852.0,
 549.0,
 276.0,
 845.0,
 810.0,
 497.0,
 461.0,
 118.0,
 703.0,
 858.0,
 371.0,
 360.0,
 209.0,
 693.0,
 191.0,
 167.0,
 710.0,
 637.0,
 579.0,
 424.0,
 314.0,
 763.0,
 838.0,
 642.0,
 820.0,
 331.0,
 692.0,
 785.0,
 674.0,
 683.0,
 519.0,
 85.0,
 524.0,
 546.0,
 802.0,
 224.0,
 92.0,
 307.0,
 834.0,
 847.0,
 274.0,
 157.0,
 474.0,
 565.0,
 813.0,
 534.0,
 377.0,
 610.0,
 149.0,
 384.0,
 293.0,
 592.0,
 545.0,
 617.0,
 507.0,
 532.0,
 885.0,
 198.0,
 398.0,
 413.0,
 857.0,
 790.0,
 566.0,
 368.0,
 791.0,
 357.0,
 771.0,
 143.0,
 694.0,
 408.0,
 106.0,
 43